<a href="https://colab.research.google.com/github/oodlbee/drug_prediction/blob/bob2708_branch/drug_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit

In [2]:
!pip install catboost

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_percentage_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

In [4]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Предобработка

## Уже обработанное

In [77]:
full = pd.read_csv("https://raw.githubusercontent.com/oodlbee/drug_prediction/master/processed_df.csv")
# full = pd.read_csv("https://raw.githubusercontent.com/oodlbee/drug_prediction/antons_branch/processed_df_without_corr.csv")
full.head(1)

,Title,"IC50, mmg/ml","CC50-MDCK, mmg/ml",SI,Molecular weight,Hydrogen bond acceptors,Hydrogen bond donors,Polar SA,SMILES,#stars,...,SAfluorine,SAamideO,PSA,#NandO,RuleOfFive,#ringatoms,#in34,#in56,#noncon,#nonHatm
0,1007-Ya-213,2.7,500.0,185.185185,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,2.0,...,0.0,0.0,35.245,2.0,0.0,0.0,0.0,0.0,0.0,14.0


In [21]:
# processed_smiles = pd.DataFrame(columns=Descriptors.CalcMolDescriptors(Chem.MolFromSmiles(full['SMILES'][0])).keys())
# for i in range(full.shape[0]):
#     processed_smiles.loc[i] = Descriptors.CalcMolDescriptors(Chem.MolFromSmiles(full['SMILES'][i])).values()

[20:54:17] Conflicting single bond directions around double bond at index 55.
[20:54:17]   BondStereo set to STEREONONE and single bond directions set to NONE.


In [78]:
def mol_dsc_calc(mols):
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in mols)

# список конституционных и физико-химических дескрипторов из библиотеки RDKit
descriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
               "NHOHCount": Descriptors.NHOHCount,
               "NOCount": Descriptors.NOCount,
               "NumHAcceptors": Descriptors.NumHAcceptors,
               "NumHDonors": Descriptors.NumHDonors,
               "NumHeteroatoms": Descriptors.NumHeteroatoms,
               "NumRotatableBonds": Descriptors.NumRotatableBonds,
               "NumValenceElectrons": Descriptors.NumValenceElectrons,
               "NumAromaticRings": Descriptors.NumAromaticRings,
               "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
               "RingCount": Descriptors.RingCount,
               "MW": Descriptors.MolWt,
               "LogP": Descriptors.MolLogP,
               "MR": Descriptors.MolMR,
               "TPSA": Descriptors.TPSA}

# sklearn трансформер для использования в конвейерном моделировании
descriptors_transformer = FunctionTransformer(mol_dsc_calc)
processed_smiles = descriptors_transformer.transform(full['SMILES'])
processed_smiles

[21:13:02] Conflicting single bond directions around double bond at index 55.
[21:13:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:13:02] Conflicting single bond directions around double bond at index 55.
[21:13:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:13:02] Conflicting single bond directions around double bond at index 55.
[21:13:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:13:02] Conflicting single bond directions around double bond at index 55.
[21:13:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:13:02] Conflicting single bond directions around double bond at index 55.
[21:13:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:13:02] Conflicting single bond directions around double bond at index 55.
[21:13:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:13:02] Conflicting single bond direc

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,14,1,2,2,1,2,2,80,0,0,2,195.306,2.26590,58.6168,32.59
1,18,0,2,2,0,2,5,104,0,0,2,250.430,3.61540,79.3190,15.60
2,16,0,2,2,0,2,3,92,0,0,2,222.376,2.83520,70.0850,15.60
3,17,0,3,3,0,3,3,98,0,0,2,239.359,2.30600,67.6630,29.54
4,41,0,4,2,0,4,16,236,0,0,4,570.995,8.44430,179.0658,24.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,18,0,3,3,0,3,7,102,1,0,1,252.354,3.71572,73.2770,39.44
1313,17,1,3,3,1,3,6,96,1,0,1,238.327,3.41272,68.3898,50.44
1314,17,0,3,3,0,3,6,96,1,0,1,238.327,3.32562,68.6600,39.44
1315,16,0,3,3,0,3,5,90,1,0,1,224.300,2.93552,64.0430,39.44


In [79]:
full.drop(["Title","SMILES"], axis=1, inplace=True)
full = full.join(processed_smiles)
full.dropna(inplace=True) # for all descriptors work
X = full.iloc[:, 3:]
y = full.iloc[:, :3]
y_si = y["SI"]
y_ic = y["IC50, mmg/ml"]
y_cc = y["CC50-MDCK, mmg/ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y_cc,  test_size=0.25, random_state=42)

In [80]:
X

,Molecular weight,Hydrogen bond acceptors,Hydrogen bond donors,Polar SA,#stars,#amine,#amidine,#acid,#amide,#rotor,...,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,195.307,2,1,32.59,2.0,0.0,0.0,0.0,0.0,10.0,...,2,2,80,0,0,2,195.306,2.26590,58.6168,32.59
1,250.431,1,0,15.60,2.0,1.0,0.0,0.0,0.0,12.0,...,2,5,104,0,0,2,250.430,3.61540,79.3190,15.60
2,222.377,1,0,15.60,3.0,1.0,0.0,0.0,0.0,10.0,...,2,3,92,0,0,2,222.376,2.83520,70.0850,15.60
3,239.361,2,0,29.54,1.0,1.0,0.0,0.0,0.0,9.0,...,3,3,98,0,0,2,239.359,2.30600,67.6630,29.54
4,570.997,2,0,24.72,7.0,0.0,0.0,0.0,0.0,30.0,...,4,16,236,0,0,4,570.995,8.44430,179.0658,24.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,252.357,3,0,39.44,1.0,0.0,0.0,0.0,0.0,7.0,...,3,7,102,1,0,1,252.354,3.71572,73.2770,39.44
1313,238.330,3,1,50.44,0.0,0.0,0.0,0.0,0.0,7.0,...,3,6,96,1,0,1,238.327,3.41272,68.3898,50.44
1314,238.330,3,0,39.44,1.0,0.0,0.0,0.0,0.0,6.0,...,3,6,96,1,0,1,238.327,3.32562,68.6600,39.44
1315,224.303,3,0,39.44,1.0,0.0,0.0,0.0,0.0,5.0,...,3,5,90,1,0,1,224.300,2.93552,64.0430,39.44


## Первый лист 1400

In [24]:
# df_1400 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/1400.xlsx')
# df_1400.head()

In [25]:
# df_1400.describe()

In [26]:
# df_1400_easy = df_1400.drop(["Title", "Pictures", "SMILES"], axis=1)
# df_1400_easy['SI'] = df_1400_easy['CC50-MDCK, mmg/ml']/df_1400_easy['IC50, mmg/ml']
# df_1400_easy.info()

In [27]:
# X = df_1400_easy.iloc[:, 3:]
# y = df_1400_easy.iloc[:, :3]
# X

In [28]:
# # Варианты целевой колонки
# y_si = y["SI"]
# y_ic = y["IC50, mmg/ml"]
# y_cc = y["CC50-MDCK, mmg/ml"]

In [29]:
# # Не забываем заменить агрумент 'y' при выборе другого целевого столбца
# X_train, X_test, y_train, y_test = train_test_split(X, y_ic,  test_size=0.25, random_state=42)

## Второй лист 1400

In [30]:
# df_1400_big = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/1400.xlsx', 1)
# df_1400_big.info()

In [31]:
# df_1400_big = df_1400_big.drop(['molecule', 'Pictures', 'SMILES', 'Unnamed: 55', 'Unnamed: 56'], axis=1)
# df_1400_big.dropna(inplace=True)

In [32]:
# X = df_1400_big.iloc[:, 3:]
# y = df_1400_big.iloc[:, :3]
# y_si = y["SI"]
# y_ic = y["IC50, mmg/ml"]
# y_cc = y["CC50-MDCK, mmg/ml"]

In [33]:
# X_train, X_test, y_train, y_test = train_test_split(X, y_cc,  test_size=0.25, random_state=42)

# Запуск моделей

## Линейная регрессия

In [81]:
lasso = LassoCV(eps = 0.001, n_alphas = 100, cv = 4, random_state = 13).fit(X_train, y_train)
# plt.figure(figsize=(10, 10), dpi=80)
# coefs = np.append(lasso.coef_, lasso.intercept_)
# names = list(X_train.columns)
# names.append('Intercept')
# g = sns.barplot(y=names, x=coefs)#, columns=X_train.columns)
# plt.show()

In [82]:
ridge = RidgeCV(alphas = np.linspace(10, 25, 100), cv = 4).fit(X_train, y_train)
# plt.figure(figsize=(10, 10), dpi=80)
# coefs = np.append(ridge.coef_, ridge.intercept_)
# names = list(X_train.columns)
# names.append('Intercept')
# g = sns.barplot(y=names, x=coefs)#, columns=X_train.columns)
# plt.show()

In [83]:
lr = LinearRegression().fit(X_train, y_train)
# plt.figure(figsize=(10, 10), dpi=80)
# coefs = np.append(lr.coef_, lr.intercept_)
# names = list(X_train.columns)
# names.append('Intercept')
# g = sns.barplot(y=names, x=coefs)#, columns=X_train.columns)
# plt.show()

## SVR

In [84]:
# line_param = np.linspace(0, 20, 20)
# parameters = {'kernel':('rbf', "poly", "sigmoid"), 'C':line_param}
# svr = make_pipeline(StandardScaler(), SVR())
# grid_search_svm = GridSearchCV(estimator=svr, param_grid=parameters, cv = 6)
# grid_search_svm.fit(X_train, y_train)
# svr = grid_search_svm.best_estimator_
svr = SVR().fit(X_train, y_train)
# svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# svr.fit(X, y)

## Деревья (лес)

In [85]:
rf = RandomForestRegressor().fit(X_train, y_train)
# rf.score(X_test, y_test)

## Деревья (Бустинг)

In [86]:
# ada = AdaBoostRegressor().fit(X_train, y_train)
cat = CatBoostRegressor(4000).fit(X_train, y_train, early_stopping_rounds=10)

Learning rate set to 0.013238
0:	learn: 130.8026227	total: 14.3ms	remaining: 57.3s
1:	learn: 130.3907741	total: 27ms	remaining: 54s
2:	learn: 129.9076931	total: 42.4ms	remaining: 56.5s
3:	learn: 129.4710466	total: 58.9ms	remaining: 58.8s
4:	learn: 129.0094975	total: 80.8ms	remaining: 1m 4s
5:	learn: 128.6206369	total: 102ms	remaining: 1m 7s
6:	learn: 128.2010498	total: 120ms	remaining: 1m 8s
7:	learn: 127.7693438	total: 139ms	remaining: 1m 9s
8:	learn: 127.3535944	total: 155ms	remaining: 1m 8s
9:	learn: 126.9692303	total: 179ms	remaining: 1m 11s
10:	learn: 126.5870550	total: 204ms	remaining: 1m 13s
11:	learn: 126.2337994	total: 227ms	remaining: 1m 15s
12:	learn: 125.8166594	total: 249ms	remaining: 1m 16s
13:	learn: 125.4085718	total: 267ms	remaining: 1m 16s
14:	learn: 125.0188930	total: 301ms	remaining: 1m 19s
15:	learn: 124.6771386	total: 323ms	remaining: 1m 20s
16:	learn: 124.3351500	total: 344ms	remaining: 1m 20s
17:	learn: 123.9772247	total: 363ms	remaining: 1m 20s
18:	learn: 123.6

# Оценка моделей

In [87]:
def calc_metrics(model, X_test=X_test, y_test=y_test, df=None):
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    r2 = model.score(X_test, y_test)
    pred_train = model.predict(X_train)
    mae_train = mean_absolute_error(y_train, pred_train)
    r2_train = model.score(X_train, y_train)
    if df is not None:
        df.loc[str(model)] = [mae_train, mae, r2_train, r2]
    else:
        print(f"R-score: {r2}\nMAE: {mae}")

In [88]:
models = [lr, ridge, lasso, svr, rf,  cat]
metrics = pd.DataFrame(columns=('MAE train', 'MAE test', 'R2 train', 'R2 test'))
for model in models:
    calc_metrics(model, df=metrics)
metrics

,MAE train,MAE test,R2 train,R2 test
LinearRegression(),78.461893,83.831885,0.420086,0.195539
"RidgeCV(alphas=array([10. , 10.15151515, 10.3030303 , 10.45454545, 10.60606061,\n 10.75757576, 10.90909091, 11.06060606, 11.21212121, 11.36363636,\n 11.51515152, 11.66666667, 11.81818182, 11.96969697, 12.12121212,\n 12.27272727, 12.42424242, 12.57575758, 12.72727273, 12.87878788,\n 13.03030303, 13.18181818, 13.33333333, 13.48484848, 13.63636364,\n 13.78787879, 13.93939394, 14.09090909, 1...\n 20.60606061, 20.75757576, 20.90909091, 21.06060606, 21.21212121,\n 21.36363636, 21.51515152, 21.66666667, 21.81818182, 21.96969697,\n 22.12121212, 22.27272727, 22.42424242, 22.57575758, 22.72727273,\n 22.87878788, 23.03030303, 23.18181818, 23.33333333, 23.48484848,\n 23.63636364, 23.78787879, 23.93939394, 24.09090909, 24.24242424,\n 24.39393939, 24.54545455, 24.6969697 , 24.84848485, 25. ]),\n cv=4)",80.168341,81.858416,0.399271,0.264469
"LassoCV(cv=4, random_state=13)",91.088635,90.316388,0.249874,0.193327
SVR(),101.625947,98.303556,-0.054707,-0.066208
RandomForestRegressor(),26.427381,68.710813,0.921903,0.407229
<catboost.core.CatBoostRegressor object at 0x7994e6173220>,14.246357,64.432464,0.978234,0.427961


In [89]:
opa = pd.DataFrame({"Predicted": cat.predict(X_test), "True": y_test})
opa.describe()

,Predicted,True
count,330.000000,330.000000
mean,140.370584,130.648788
std,93.037353,122.894382
min,-6.132925,0.100000
25%,62.947689,30.425000
50%,113.759126,80.400000
75%,217.616957,300.000000
max,415.739868,500.000000


In [94]:
si = opa2/opa
# mean_absolute_error(si.iloc[:, 0], si.iloc[:, 1])
mean_absolute_error(opa.iloc[:, 0], opa.iloc[:, 1])


46.68310240904196